## Constants in Pluto-Eris

repo: https://github.com/privacy-scaling-explorations/halo2curves/tree/main/src/pluto_eris , https://github.com/daira/pluto-eris \
paper: https://eprint.iacr.org/2010/354.pdf

### 1. Helper functions

Given an integer $ 0 < n < 2^{448}$, return its 7 limbs in little-endian where each 64-bit limb is in big endian.

In [1]:
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

Given an integer $x$, compute its little endian NAF whose Hamming weight is optimal. \
Definition of NAF: https://en.wikipedia.org/wiki/Non-adjacent_form

In [2]:
def naf(n):
    bits = []
    while n > 0:
        if n % 2 == 1:
            # If n is odd, set the current digit as 2 - (n mod 4).
            digit = 2 - (n % 4)
            n -= digit
        else:
            # If n is even, set the current digit as 0.
            digit = 0
        bits.append(digit)
        n //= 2
    return bits

# Example:
# decimal_number = 7
# naf = naf(decimal_number)
# print(naf): [-1, 0, 0, 1], the binary form is initially [1, 1, 1] = 1 + 2 + 4.

In [3]:
# output the indices and the corresponding non-zero bits for a given NAF
def non_zero_bits(naf):
    l = len(naf)
    value = 0
    hamming_bits = {}
    for i in range(l):
        if naf[i] != 0:
            hamming_bits[i] = naf[i]
            value = value + naf[i] * 2**i
    # sanity check
    new_value = 0
    for index, bit in hamming_bits.items():
        new_value = new_value + bit * 2**index
    assert new_value == value, "the decomposition is not correct"
    return hamming_bits

In [4]:
print(non_zero_bits(naf(7)))

{0: -1, 3: 1}


### 2. The Base Field $\mathbb{F}_p$
seed $U = -1298074214633708060054710657220608$ \
the base field modulus $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$ \
we define the constant $NEG\_U = -U$:

In [5]:
U = -1298074214633708060054710657220608
NEG_U = -U
NEG_U.bit_length()

111

In [6]:
hex(NEG_U)

'0x4000000000001000008780000000'

In [7]:
print(non_zero_bits(naf(NEG_U)))

{31: -1, 35: 1, 39: 1, 60: 1, 110: 1}


A low-hamming weight representation of $-U$ is thus:
$$-U = 2^{110} + 2^{60} + 2^{39} + 2^{35} - 2^{31}$$

As the base field size $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$, the low Hamming-weight representation of $p$ using symbolic computation is:

In [8]:
R_over_q = PolynomialRing(QQ, 't')  # 't' stands for two
t = R_over_q.gen() # Don't forget to explicit the generator of ring
poly_U = -(t^(110) + t^(60) + t^(39) + t^(35) - t^(31))
# p = 9 * 2^2 * U^4 + 9 * 2^2 * U^3 + 3 * 2^3 * U^2 + 3 * 2^1 * U + 1
poly_p = 9 * t^2 * poly_U^4 + 9 * t^2 * poly_U^3 + 3 * t^3 * poly_U^2 + 3 * t * poly_U + 1
poly_p

9*t^442 + 36*t^392 + 36*t^371 + 36*t^367 - 36*t^363 + 54*t^342 - 9*t^332 + 108*t^321 + 108*t^317 - 108*t^313 + 54*t^300 + 108*t^296 - 18*t^292 - 108*t^288 + 54*t^284 - 27*t^282 + 108*t^271 + 108*t^267 - 108*t^263 - 27*t^261 - 27*t^257 + 27*t^253 + 108*t^250 + 216*t^246 - 99*t^242 - 216*t^238 + 108*t^234 - 27*t^232 + 36*t^229 + 108*t^225 + 3*t^223 + 36*t^221 - 144*t^217 - 36*t^213 - 54*t^211 + 108*t^209 - 54*t^207 - 36*t^205 + 54*t^203 + 54*t^200 + 108*t^196 - 54*t^192 - 27*t^190 - 108*t^188 - 54*t^186 + 54*t^184 + 18*t^182 + 36*t^179 + 54*t^178 + 108*t^175 - 27*t^174 + 6*t^173 - 180*t^167 - 27*t^161 + 108*t^159 + 9*t^158 - 27*t^157 - 36*t^155 + 36*t^154 + 27*t^153 + 6*t^152 + 18*t^150 + 6*t^148 - 72*t^146 - 6*t^144 - 45*t^142 - 27*t^140 + 72*t^138 - 54*t^136 + 18*t^134 + 27*t^132 - 36*t^130 + 54*t^128 + 9*t^126 - 27*t^124 + 3*t^123 - 9*t^119 - 27*t^115 - 3*t^111 + 45*t^107 + 6*t^102 - 27*t^99 + 6*t^98 + 9*t^95 - 6*t^94 + 3*t^81 + 6*t^77 - 3*t^73 - 6*t^69 + 3*t^65 - 3*t^61 - 3*t^40 - 3*

Therefore, $p = 9 \cdot 2^{442} + 9 \cdot 2^{394} + 9 \cdot 2^{373} + 9 \cdot 2^{369} + \dots$ thus its hamming weight is large. This is a 
serious constraint for applying some optimizations over the base field $\mathbb{F}_{p}$ (e.g Fiat-Crypto).

The constant $- (6U + 2)$

In [9]:
NEG_SIX_U_PLUS_2 = -(6 * U + 2)

In [10]:
hex(NEG_SIX_U_PLUS_2)

'0x18000000000006000032cfffffffe'

NAF of the value $-(6U + 2)$, refer to
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22-L27

In [11]:
print(naf(NEG_SIX_U_PLUS_2))

[0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, -1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1]


The length of the list used to store NEG_SIX_U_PLUS_2 \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22


In [12]:
len(naf(NEG_SIX_U_PLUS_2))

114

In [13]:
print(non_zero_bits(naf(NEG_SIX_U_PLUS_2)))

{1: -1, 32: 1, 34: -1, 36: -1, 38: 1, 40: -1, 42: 1, 61: -1, 63: 1, 111: -1, 113: 1}


Due to the existence of $-1$, the length of NAF is bigger than the usual bit length:

In [14]:
NEG_SIX_U_PLUS_2.bit_length()

113

The base field modulus $p$ in terms of $U$:

In [15]:
p = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(p)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001'

In [16]:
p

102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

In [17]:
p.bit_length()

446

Check the group order $p-1$ is divisible by $6$ and $4$

In [18]:
(p - 1) % 6 == 0

True

In [19]:
(p - 1) % 4 == 0

True

The limbs representation of $p$, refer to:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L39-L47

In [20]:
u_64_little_endian(p)

['0x9ffffcd300000001',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

Construct the base field $\mathbb{F}_p$

In [21]:
Fp = GF(p)
Fp

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

The constant $R$ for Montgomery form, refer to
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L67-L75

In [22]:
u_64_little_endian((2**448) % p)

['0xa000163afffffff9',
 '0x8d68a2aaffd0ef18',
 '0xbf6a760a123e0121',
 '0x2242c7760637089c',
 '0x67e576bf526ff2f5',
 '0xf7a9dfffa183e9bf',
 '0x03ffffffffff03ff']

In [23]:
R = Fp(2**448)
R

11356855067116315761326349333718857071609919219953404605758555192204529273465116571178922486933671965396447230942486297326349317046265

### 3. The Scalar Field $\mathbb{F}_q$
the scalar field modulus $q = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [24]:
q = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(q)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001'

The limbs representation of $q$, refer to:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fq.rs#L39-L47

In [25]:
u_64_little_endian(q)

['0x1ffffcd300000001',
 '0x9ca7e85d60050af4',
 '0xe4a775fe8e177fd6',
 '0x443f9a5c7a8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [26]:
Fq = GF(q)
Fq

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855480569854275933742834077002685857629445612735086326265689167708028929

### 4. The extension fields
The field $\mathbb{F}_{p^2}$ 
defined by $\mathbb{F}_{p^2} = \mathbb{F}_p[u] = \mathbb{F}_p[X] / (X^2 + 5)$ where $X^2 + 5$ is irreducible in $\mathbb{F}_p[X]$\
refer to: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp2.rs

Check $X^2 + 5$ is irreducible:

In [27]:
Rp = PolynomialRing(Fp, "x")
x = Rp.gen()
(x^2 + 5).is_irreducible()

True

Construct $\mathbb{F}_{p^2}$ from $\mathbb{F}_{p}$ in $u$ such that $u^2 + 5 = 0$

In [28]:
Fp2.<u> = Fp.extension(x^2 + 5)
Fp2

Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2

Check the identity $u^2 + 5 = 0$ in $\mathbb{F}_{p^{2}}$

In [29]:
u**2 + 5 == 0

True

The twisted curve (Triton) defined by: $E': Y^2 = X^3 + (u + 3)$ \
see the defintion in: https://github.com/daira/pluto-eris \
and in the repo: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/curve.rs#L100-L106


The field $\mathbb{F}_{p^6}$, refer to: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs\
$\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[v] = \mathbb{F}_{p^2}[X] / (X^3 - 57/(u + 3))$ where $X^3 - 57/(u+3)$ is irreducible in $\mathbb{F}_{p^2}[X]$\

Note that the non-cubic residue $57/(u + 3)$ is not arbitrarily chosen. It is closely related to the equation of the curve:
$$E / \mathbb{F}_p: Y^2 = X^3 + 57$$
and the twisted curve :
$$E' / \mathbb{F}_{p^2}: Y^2 = X^3 + (u + 3)$$
More precisely, let $\xi = 57/(u + 3) \in \mathbb{F}_{p^2}$, the isomorphism is defined by
$$\Phi: E' \rightarrow E: \quad (x, y) \mapsto (x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
and
$$\Phi^{-1}: E \rightarrow E': \quad (x, y) \mapsto (x / \xi^{1/3}, y / \xi^{1/2})$$
See more details in Section 3 regarding D-type twist in paper: http://indigo.ie/~mscott/twists.pdf

Suppose $\xi = 57/(u + 3) = c_1 \cdot u + c_0$ where $c_0, c_1 \in \mathbb{F}_p$

In [30]:
xi = 57 * (u + 3)**(-1)
xi

21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763*u + 36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624

$\xi^{(p^2 - 1)/2 }$ is order $2$

In [31]:
xi**((p ** 2 - 1)/2) == Fp2(-1)

True

The coordinate $c_0$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$ (copied from the above result), refer to\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L23-L32

In [32]:
u_64_little_endian(36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624)

['0xddb6da4b5b6db6e8',
 '0x833bf7b35b701d98',
 '0x3f6072240ebe2483',
 '0x73cd928ee056022c',
 '0xce4a7f2a7bcb4495',
 '0xdbda9924971b3a9a',
 '0x0cdb6db6db6dc3b6']

The coordinate $c_1$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$ (copied from the above result), refer to\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L33-L42

In [33]:
u_64_little_endian(21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763)

['0xeb6db62d36db6db3',
 '0xb523fb0536dcde8e',
 '0x8c6d1148d5a5491b',
 '0x457b57ef5366ce1a',
 '0x489319197d79f5f3',
 '0xb71cc2492776bcc3',
 '0x07b6db6db6db756d']

The irreducible polynomial in $\mathbb{F}_{p^2}[X]$: $X^3 - \xi$

In [34]:
Rp2 = PolynomialRing(Fp2, "x")
x = Rp2.gen()
(x^3 - xi).is_irreducible()

True

Construct the field $\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[X]/(X^3 - \xi) = \mathbb{F}_{p^2}[v]$

In [35]:
Fp6.<v> = Fp2.extension(x^3 - xi)
Fp6

Univariate Quotient Polynomial Ring in v over Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2 with modulus v^3 + 80309189403197921659877265756093290307166794450418481759520898815028391298359804839848300107093151321928845082944495999982558932931150*u + 65707518602616481358081399254985419342227377277615121439608008121386865607748931232603154633076214717941782340590951272713002763307289

In [36]:
Fp6.is_field()

True

Check the identity $v^3 - \xi = 0$

In [37]:
v**3 - xi == 0

True

The field $\mathbb{F}_{p^{12}} = \mathbb{F}_{p^6}[w] = \mathbb{F}_{p^6}[X] / (X^2 - v)$ where $X^2 - v$ is irreducible in $\mathbb{F}_{p^6}[X]$ \
We may use the following constructor to construct $\mathbb{F}_{p^{12}}$ as usual, but it turns out too slow:


In [38]:
# Rp6 = PolynomialRing(Fp6, "x")
# x = Rp6.gen()
# (x^2 - v).is_irreducible()
# Fp12 = Rp6.quotient(x^2 - v)
# Fp12

Instead, let's check it manually. It suffices to prove $X^2 - v$ is irreducible over $\mathbb{F}_{p^6}[X]$, in other words
$v$ is not a quadratic residue over $\mathbb{F}_{p^6}$. Otherwise, there is an element $p(v) = a_0 + a_1 \cdot v + a_2 \cdot v^2 \in \mathbb{F}_{p^6}$ such that
$$p(v)^2 = v \text{ over } \mathbb{F}_{p^6}$$
Equivalently, there exists $p(X) = a_0 + a_1 \cdot X + a_2 \cdot X^2 \in \mathbb{F}_{p^2}[X]$ such that
$$ X^3 - 57/(u + 3) \;| \; p(X)^2 - X \text{ over } \mathbb{F}_{p^2}[X]$$
It is clear that there is no such a $p(X) \in \mathbb{F}_{p^2}[X]$.


### 5. The constants in Pairing
The const FROBENIUS_COEFF_FP6_C1[1] 
used in pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L462

In fact, the list FROBENIUS_COEFF_FQ6_C1 = $[\gamma_{10}. \gamma_{11}, \dots, \gamma_{15}]$ such that for the Frobenius morphism $\pi$:
$$\pi^i(v) = \gamma_{1i} \cdot v \quad \text{ for } i = 0, 1, \dots, 5$$
where $\pi$ has order $6$ in the group $Gal(\mathbb{F}_{q^6} / \mathbb{F}_q)$ and $\mathbb{F}_{q^6} = \mathbb{F}_{q^{2}}[v]$.

Notice $\{ 1, v, v^2\}$ is a basis for the $\mathbb{F}_{q^2}$-vector space $\mathbb{F}_{q^6}$, regarding the images of $v^2$, the list FROBENIUS_COEFF_FQ6_C2 = $[\gamma_{20}. \gamma_{21}, \dots, \gamma_{25}]$ is defined by:
$$\pi^i(v^2) = \gamma_{2i} \cdot v^2 \quad \text{ for } i = 0, 1, \dots, 5$$

For $v = (57/(u + 3))^{1/3} = \xi^{1/3}$, we have
$$\pi(v) = \xi^{p/3} = \xi^{(p-1)/3} \cdot \xi^{1/3} = \xi^{(p-1)/3} \cdot v$$
hence $\gamma_{11} = \xi^{(p-1)/3}$.

This constant is defined: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L285

In [39]:
xi**((p-1)/3)

92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858*u + 51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611

Let $\gamma_{11} = \xi^{(p-1)/3} = c_1 \cdot u + c_0 \in \mathbb{F}_{p^2}$\
the coordinate $c_0$: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L287-L296 \
The representation from BN256 is in the field representation (with Montgomery const $R$).


In [40]:
hex(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

'0x120de97f024c55bc3bc0d351f4c70da1e3886170077a50986f93678bc921dcd5041bc4bb14cc42dc52e787634eccc335a001825382850d03'

In [41]:
u_64_little_endian(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

['0xa001825382850d03',
 '0x52e787634eccc335',
 '0x041bc4bb14cc42dc',
 '0x6f93678bc921dcd5',
 '0xe3886170077a5098',
 '0x3bc0d351f4c70da1',
 '0x120de97f024c55bc']

The coordinate $c_1$: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L297-L306

In [42]:
hex(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

'0x2096f3f804d973afd82becc2ef081b76132461908eadbe3da1a7f5502b7091965efa1ddf4658080413be1b7cd3c9ea0e2772fea378a9b322'

In [43]:
u_64_little_endian(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

['0x2772fea378a9b322',
 '0x13be1b7cd3c9ea0e',
 '0x5efa1ddf46580804',
 '0xa1a7f5502b709196',
 '0x132461908eadbe3d',
 '0xd82becc2ef081b76',
 '0x2096f3f804d973af']

The const FROBENIUS_COEFF_FP6_C1[2] 
used in pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L470

By the above discussion, we need to compute the constant $\gamma_{12}$ such that $\pi^2(v) = \gamma_{12} \cdot v$. It is clear
$$\pi \circ \pi(v) = \pi(\gamma_{11} \cdot v) = \overline{\gamma}_{11} \cdot \pi(v) = \overline{\gamma}_{11} \cdot \gamma_{11} \cdot v$$
since $\gamma_{11} \in \mathbb{F}_{p^2}$ and the Frobenius over $\mathbb{F}_{p^2}$ is equivalent to conjugation. Hence
$\gamma_{12} = \overline{\gamma}_{11} \cdot \gamma_{11} = \xi^{((p^2 - p) + p - 1)/3} = \xi^{(p^2 - 1)/3}$.

This constant is defined: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L308

In [44]:
xi**((p**2-1)/3)

39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334

$c_1 = 0$ and $c_0$ is referred to https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fp6.rs#L310-L319

In [45]:
hex(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

'0x480000000000360001c950000d7ee0e4a803c956d01c903d720dc8ad8b38dffaf50c100004c37ffffffe'

In [46]:
u_64_little_endian(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

['0x100004c37ffffffe',
 '0xc8ad8b38dffaf50c',
 '0xc956d01c903d720d',
 '0x50000d7ee0e4a803',
 '0x00000000360001c9',
 '0x0000000000004800',
 '0x0000000000000000']

The const XI_TO_P_MINUS_1_OVER_2, 
used in pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L29-L50


In [47]:
xi**((p-1)/2)

65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996*u + 95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166

the coordinate $c_0$

In [48]:
u_64_little_endian(95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166)

['0x54cf5ad1c0926216',
 '0x186c1f3ce4a46d4e',
 '0x9c23800ce9c9452f',
 '0x50e0d09ff6d6c08b',
 '0x7cf421e4d46f6666',
 '0x678664ba4b6d8343',
 '0x21cc26d5de0f80f4']

The coordinate $c_1$

In [49]:
u_64_little_endian(65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996)

['0xc0505f4c260e91f4',
 '0xe7bbd15f10723657',
 '0xb4b3e0c35358097e',
 '0x87c56f42a558750d',
 '0x4b7211d23f34f0ae',
 '0xf6839d29e2f0d250',
 '0x16ebe8b2e12a1106']

The order of $\xi$ is exactly $p^2 - 1$, thus $\xi^{(p^2 - 1)/2} = 1$. This fact is used in $y$ coordinate of 
the point $ -Q_2 = - \pi^2(Q)$.

In [50]:
xi**((p**2 - 1)/2) == -1

True

Rationale of These Constants: \
These constants are used for computing $Q_1 = \pi(Q)$ and $Q_2 = \pi^2(Q)$.\
Recall in the implementation, $Q \in G_2$  as an affine point in twisted curve $E'(\mathbb{F}_{p^2})$. Suppose $Q = (x, y) \in E'(\mathbb{F}_{p^2})$, then
$$\Phi^{-1}: E' \longrightarrow E: (x, y) \mapsto (x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
gives preimage of $Q$ in $E(\mathbb{F}_{p^{12}})$
$$(x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
We then apply Frobenius in $E(\mathbb{F}_{p^{12}})$ as usual and map back to the twisted curve $E'(\mathbb{F}_{p^2})$:
$$E(\mathbb{F}_{p^{12}}) \xrightarrow{\pi} E(\mathbb{F}_{p^{12}}) \xrightarrow{\Phi^{-1}} E'(\mathbb{F}_{p^{2}})$$
$$(x \cdot \xi^{1/3}, y \cdot \xi^{1/2}) \mapsto (\overline{x} \cdot \xi^{p/3}, \overline{y} \cdot \xi^{p/2}) \mapsto (\overline{x} \cdot \xi^{(p-1)/3}, \overline{y} \cdot \xi^{(p-1)/2})$$
So $Q_1 = (\overline{x} \cdot \xi^{(p-1)/3}, \overline{y} \cdot \xi^{(p-1)/2})$, justified by: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L461-L467

### 6. The group of the curve Pluto

The equation of Pluto:
$$E / \mathbb{F}_{p}: y^2 = x^3 + 57$$

In [51]:
E = EllipticCurve(Fp, [0, 57])
E

Elliptic Curve defined by y^2 = x^3 + 57 over Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

The order of the group $E(\mathbb{F}_{p})$ is $q$ which is prime, thus the group is cyclic and $E(\mathbb{F}_{p}) = \mathbb{G}_{1}$

In [52]:
E.cardinality() == q

True

In [53]:
E.abelian_group()

Additive abelian group isomorphic to Z/102211695604070082112571065507755096754575920209623522239390234855480569854275933742834077002685857629445612735086326265689167708028929 embedded in Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 57 over Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

We take the generator of $\mathbb{G}_{1}$ as $(-2, 7)$, as fixed in Daira's repo

In [54]:
G1_GENERATOR_X = Fp(-2)
G1_GENERATOR_Y = Fp(7)
G1_GENERATOR = E(G1_GENERATOR_X, G1_GENERATOR_Y)
G1_GENERATOR.order() == q

True

In [55]:
G1_GENERATOR_X

102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366911

We test the values with the ones given in the repo: https://github.com/daira/pluto-eris

In [56]:
u_64_little_endian(p) == [
    '0x9ffffcd300000001',
    '0xa2a7e8c30006b945',
    '0xe4a7a5fe8fadffd6',
    '0x443f9a5cda8a6c7b',
    '0xa803ca76f439266f',
    '0x0130e0000d7f70e4',
    '0x2400000000002400',
]

True

In [57]:
u_64_little_endian(Integer(G1_GENERATOR_X))

['0x9ffffcd2ffffffff',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [58]:
u_64_little_endian(Integer(G1_GENERATOR_Y))

['0x0000000000000007',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000']

The twist curve Triton, i.e $\mathbb{G}_2$, defined by
$$E_{2} / \mathbb{F}_{p^{2}}: y^{2} = x^{3} + (u + 3)$$ 
where $\mathbb{F}_{p^2} = \mathbb{F}_{p}[u]$ such that $u^2 + 5 = 0$.

In [59]:
E2 = EllipticCurve(Fp2, [0, u + 3])
E2

Elliptic Curve defined by y^2 = x^3 + (u+3) over Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2

The order of the group $\mathbb{G}_2 = E_{2}(\mathbb{F}_{p^{2}})$ is $q (2 p - q)$ as showed in the repo https://github.com/daira/pluto-eris, 
we know this group is also cyclic based on elliptic curves theory

In [60]:
E2.cardinality() == q * (2 * p - q)

True

We take a random point $P$ in $E_{2}(\mathbb{F}_{p^{2}})$, its order is very probably $q(2p - q)$, \
then we only need to take the generator of $\mathbb{G}_{2}$ as $Q = (2p - q) \cdot P$ whose order is thus $q$ as required

In [61]:
G2_GENERATOR = E2(73825144661657802320622230235198938990499597858335392319575549414437389618818590520326285321360920202797735030655909561889777699859203*u + 71734302620089618492699913728937268183797385533150575889378098330781954797102305587510777407800704588982064101415828137491262185115028, 98071031875237312442546746862775115431449232820321509333598658952664126924349333079397199359138721588981040507328584731002107992741432*u + 38777463725275824700618213641456882570018733940119470655919893307532794266845691064759257478242590798775325761756453196673183479571667)
G2_GENERATOR

(73825144661657802320622230235198938990499597858335392319575549414437389618818590520326285321360920202797735030655909561889777699859203*u + 71734302620089618492699913728937268183797385533150575889378098330781954797102305587510777407800704588982064101415828137491262185115028 : 98071031875237312442546746862775115431449232820321509333598658952664126924349333079397199359138721588981040507328584731002107992741432*u + 38777463725275824700618213641456882570018733940119470655919893307532794266845691064759257478242590798775325761756453196673183479571667 : 1)

In [62]:
# its order is indeed q
q * G2_GENERATOR

(0 : 1 : 0)

In [63]:
# extract the coordinates of the generator g2, each of which is in Fp2
(G2_GENERATOR_X, G2_GENERATOR_Y) = G2_GENERATOR.xy()

In [64]:
# Output (c0, c1) = x in Fp2 for the generator g2 = (x, y) in Fp2 * Fp2
G2_GENERATOR_X.list()


[71734302620089618492699913728937268183797385533150575889378098330781954797102305587510777407800704588982064101415828137491262185115028,
 73825144661657802320622230235198938990499597858335392319575549414437389618818590520326285321360920202797735030655909561889777699859203]

In [65]:
# bit lengths of the coordinates of g2.x
G2_GENERATOR_X_0 = G2_GENERATOR_X.list()[0]
G2_GENERATOR_X_1 = G2_GENERATOR_X.list()[1]
(Integer(G2_GENERATOR_X_0).bit_length(), Integer(G2_GENERATOR_X_1).bit_length())

(445, 445)

In [66]:
u_64_little_endian(Integer(G2_GENERATOR_X_0))

['0x64df94ec64138d94',
 '0x9c1b62269cfa10e4',
 '0x3886dca44a7f6c19',
 '0x9b02208cfd650e17',
 '0x052896a707cf8947',
 '0xa9490edfe7707f05',
 '0x1943fb34a457551b']

In [67]:
u_64_little_endian(Integer(G2_GENERATOR_X_1))

['0xb1bd9958ca2d2b03',
 '0xf34dd1eb12904ef0',
 '0xfe10783b9106e92f',
 '0x343fc095636c8117',
 '0x8fc4095ab8d5f6c0',
 '0x93454f05c55110e5',
 '0x1a0080f48de3bb46']

In [68]:
# bit lengths of the coordinates of g2.y
G2_GENERATOR_Y_0 = G2_GENERATOR_Y.list()[0]
G2_GENERATOR_Y_1 = G2_GENERATOR_Y.list()[1]
(Integer(G2_GENERATOR_Y_0).bit_length(), Integer(G2_GENERATOR_Y_1).bit_length())

(444, 446)

In [69]:
u_64_little_endian(Integer(G2_GENERATOR_Y_0))


['0x1a40ea5e6b9924d3',
 '0xe7ad21b3504fc522',
 '0x54293e270ef1d75d',
 '0x5796b6e18f6c5e39',
 '0x20540d68e5e7cafe',
 '0xc72900d3c580b99b',
 '0x0da866bbcb2696a5']

In [70]:
u_64_little_endian(Integer(G2_GENERATOR_Y_1))

['0xcf3ae4b7c5181638',
 '0x0fd1fedc66c1f210',
 '0xaa33870a9e316d6f',
 '0xc844de8ce3795939',
 '0x2068f65c6ea24746',
 '0xe282c87cb0420ae1',
 '0x228aa759cf616115']

### 6. The constants in Endoscaling

$\zeta_{p}$ is one of the two $3nd$ primitive roots of unity in $\mathbb{F}_{p}$, the other one is $\zeta_{p}^{2}$.

In [71]:
zetap = Fp.zeta(3)
zetap

39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334

$\zeta_{q}$ is one of the two $3nd$ primitive roots of unity in $\mathbb{F}_{q}$, the other one is $\zeta_{q}^{2}$.


In [72]:
zetaq = Fq.zeta(3)
zetaq

102211695604070082112571065507755018013549828020984436483043340076454780464112937796556235142799808455295617893795351495840632510873602

The endomorphism of $E/\mathbb{F}_{p}$ defined by:
$$(x, y) \mapsto (\zeta x, y)$$
where $\zeta$ is a primitive $3rd$ root

In [73]:
def endo (E, zeta, P):
    (xp, yp) = P.xy()
    return E(zeta * xp, yp)

We know 
$$(\zeta_{p} x, y) = [\zeta_{q}] (x, y)$$
but we should find the right $\zeta_{p}$ which maybe $\zeta_{p}$ or $\zeta_{p}^2$, the same applies to $\zeta_{q}$

In [74]:
zP = endo(E, zetap, G1_GENERATOR)
if zP == int(zetaq)*G1_GENERATOR:
    print("1st trial")
else:
    zetaq = zetaq^2
    if zP == int(zetaq)*G1_GENERATOR: 
        print("2nd trial")
    else: 
        zetap = zetap^2
        zP = endo(E, zetap, G1_GENERATOR)
        if zP == int(zetaq)*G1_GENERATOR:
            print("3nd trial")
        else:
            zetaq = zetaq^2
            assert zP == int(zetaq)*G1_GENERATOR
            print("4th trial")
(zetap, zetaq)

2nd trial


(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334,
 78741026092188639085756346894779025789390162995946277841859886049174149994841290974769848535197155326)

The bit lengths of $\zeta_{p}$ and $\zeta_{q}$ are exactly as mentioned in the Pluto/Eris repo

In [75]:
(Integer(zetap).bit_length(), Integer(zetaq).bit_length())

(335, 336)

### 7. The constants in GLV method

We follow the method mentioned in:
https://hackmd.io/@drouyang/glv
and the paper:
https://www.iacr.org/archive/crypto2001/21390189.pdf

Given two integers $n$ and $\lambda$, GLV method outputs two short vectors $v_1 = (a_1, b_1), v_2 = (a_2, b_2)$ such that 
$$ a_1 + b_1 \lambda = 0\mod n $$
$$ a_2 + b_2 \lambda = 0\mod n $$

In [76]:
def ext_euclidean(a, b):
    assert a <= b
    sqr_b = sqrt(b)
    u = a
    v = b
    x1 = 1
    y1 = 0
    x2 = 0
    y2 = 1

    X = []
    Y = []
    R = []

    flag = -1
    i = -1
    while not u == 0:
        q = v // u
        r = v - q * u
        if flag == -1:
            if r < sqr_b:
                flag = i # exactly the largest index i such that R[i] >= sqrt_b
        i = i + 1
        R.append(r)  # R[i] = r
        x = x2 - q * x1
        X.append(x)
        y = y2 - q * y1  # ax + by = d
        Y.append(y)
        v = u
        u = r
        x2 = x1
        x1 = x
        y2 = y1
        y1 = y
    
    d = v
    x = x2
    y = y2
    X.append(x)
    Y.append(y)
    R.append(0)

    # ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2))
    return ((R[flag], X[flag]), (R[flag + 1], X[flag + 1]), (R[flag + 2], X[flag + 2]))  

def GLV(scalar, n):
    ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2)) = ext_euclidean(scalar, n)
    b_1 = t_l_1  # to be consistent with the repo, here we indeed assign -b_1 for the coordinate b1 of v1
    a_1 = r_l_1
    if (r_l^2 + t_l^2) <= (r_l_2^2 + t_l_2^2):
        b_2 = -t_l
        a_2 = r_l
    else:
        b_2 = -t_l_2
        a_2 = r_l_2

    return ((a_1, b_1), (a_2, b_2))  # thus the two short vectors: v_1 = (a_1, -b_1) and v_2 = (a_2, b_2)
    

In [77]:
# the scalar takes the 3rd primitive root of Fq
scalar = int(zetaq)
n = q
scalar <= n

True

In [78]:
# output the two short vectors v1, v2
((a1, b1), (a2, b2)) = GLV(scalar, n)
(u_64_little_endian(b1), u_64_little_endian(b2))

(['0x3ffffde200000001',
  '0x05ff0065a001ae51',
  '0x0000300001968000',
  '0x0000000060000000',
  '0x0000000000000000',
  '0x0000000000000000',
  '0x0000000000000000'],
 ['0x2000010effffffff',
  '0x0000800000000000',
  '0x0000000000000000',
  '0x0000000000000000',
  '0x0000000000000000',
  '0x0000000000000000',
  '0x0000000000000000'])

Compute $\gamma_1 = \lfloor -b_1 \cdot 2^{256} / r \rceil$, as $ -b_1 / r < 1$. Recall the variable $b_1$ in the code represents $-b_1$ for the coordinate $b_1$ of $v_1$. \
And $\gamma_2 = \lfloor b_2 \cdot 2^{256} / r \rceil$, where $b_2$ is the coordinate of $v_2$. \
Their counterpart of BN256  can be referred to: https://github.com/privacy-scaling-explorations/halo2curves/blob/6abfdbb0eefd476864e1ed91d32ab84c7b36c203/src/bn256/curve.rs#L127-L130 \ 
Note the comment in the repo is not correct.

In [79]:
gamma1 = round(b1 * 2^448 / q )
u_64_little_endian(gamma1)

['0xdf52222a6a19d56d',
 '0x2aae415b8e5c9500',
 '0xaaa955554a0aaaab',
 '0x00000002aaaaaaaa',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000']

In [80]:
gamma2 = round(b2 * 2^448 / q )
u_64_little_endian(gamma2)

['0xe38e224e38e4e395',
 '0x00038e38e38e38e0',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000',
 '0x0000000000000000']

Consequently, given an arbitrary scalar $k$, we use GLV method to decompose: \
$k = k_1 + k_2 \times \zeta$
where $k_1, k_2$ are almost the half bit length of $k$ (recall $k$ with $446$ bits)

To make consistent with Algo 3.74 in https://hackmd.io/@drouyang/glv, we have to clarify the code above each line in the notations used in Algo 3.74.

In [81]:
def decompose(k):
    c1_ = gamma2 * k
    c2_ = gamma1 * k
    # round(b2 * k / n)
    c1 = c1_ >> 448
    # round(-b1 * k / n)
    c2 = c2_ >> 448
    # c1 * -b1
    q1 = c1 * b1
    # c2 * b2
    q2 = c2 * b2
    # -c1 * b1 - c2 * b2
    k2 = q1 - q2
    # k - c1 * a1 - c2 * a2
    k1 = k - c1 * a1 - c2 * a2
    # the correctness of GLV involves the following constraints about bit lengths and identity
    return (k1.bit_length()<=224 and k2.bit_length() <=224 and Fq(k) == Fq(k1 + k2 * int(zetaq)))    

We use a simple test for 1000 random scalar values:

In [82]:
for i in range(1000):
    k = int(Fq.random_element())
    if decompose(k) == False:
        print("the algorithm is broken")
        break
print("pass")

pass
